<a href="https://colab.research.google.com/github/kenton-tang-kl/Exploring-the-Robustness-of-Dialogue-Based-Summarization/blob/main/Robust_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Project Setup

In [ ]:
!pip install transformers datasets evaluate rouge-score py7zr

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 32.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=7232bd3e813e109adb6954d1da0415acfad59573c5ac61ec77ccccfc5d9ad865
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Step 2: Data Preparation

In [ ]:
from datasets import load_dataset

samsum_dataset = load_dataset("samsum")

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
samsum_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
samsum_dataset['test'][0]

{'id': '13862856',
 'dialogue': "Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye",
 'summary': "Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."}

In [ ]:
xsum_dataset = load_dataset("EdinburghNLP/xsum")

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
xsum_dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [ ]:
xsum_dataset['test'][0]

{'document': 'Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.\nWorkers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.\nThe Welsh Government said more people than ever were getting help to address housing problems.\nChanges to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.\nPrison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.\nHowever, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.\nAndrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the

In [ ]:
print(samsum_dataset['train'].filter(lambda example: example['dialogue'] is None or example['summary'] is None))
print(xsum_dataset['test'].filter(lambda example: example['document'] is None or example['summary'] is None))

Filter:   0%|          | 0/14732 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 0
})


Filter:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 0
})


In [ ]:
samsum_train = samsum_dataset['train']
samsum_val = samsum_dataset['validation']
samsum_test = samsum_dataset['test']

In [ ]:
xsum_test = xsum_dataset['test']

In [ ]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def preprocess_function(example, input_key):
    inputs = tokenizer(example[input_key], max_length=1024, truncation=True, padding='max_length', return_tensors="pt")
    targets = tokenizer(example['summary'], max_length=128, truncation=True, padding='max_length', return_tensors="pt")
    return {
        'input_ids': inputs['input_ids'].squeeze(),
        'attention_mask': inputs['attention_mask'].squeeze(),
        'labels': targets['input_ids'].squeeze()
    }

In [ ]:
samsum_train = samsum_train.map(lambda x: preprocess_function(x, 'dialogue'), batched=True)
samsum_val = samsum_val.map(lambda x: preprocess_function(x, 'dialogue'), batched=True)
samsum_test = samsum_test.map(lambda x: preprocess_function(x, 'dialogue'), batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [ ]:
xsum_test = xsum_test.map(lambda x: preprocess_function(x, 'document'), batched=True)

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
samsum_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
samsum_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
samsum_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

xsum_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
xsum_test_subset = xsum_test.select(range(819))

Step 3: Baseline Evaluation

In [ ]:
from transformers import BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-cnn-12-6").to(device)

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [ ]:
def generate_summary(example):
    inputs = example['input_ids'].unsqueeze(0).to(device)
    attention_mask = example['attention_mask'].unsqueeze(0).to(device)

    summary_ids = model.generate(inputs, attention_mask=attention_mask, max_length=128, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return {"generated_summary": summary}

In [ ]:
samsum_test = samsum_test.map(generate_summary)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [ ]:
xsum_test_subset = xsum_test_subset.map(generate_summary)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [ ]:
print("Dialogue: \n", samsum_dataset['test'][0]['dialogue'])
print("Original: ", samsum_dataset['test'][0]['summary'])
print("Generated: ", samsum_test[0]['generated_summary'])

Dialogue: 
 Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
Original:  Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.
Generated:   Hannah: Hey, do you have Betty's number? I don't know him well. He called her last time we were at the park together.                Amanda: Ask Larry. The last time I've seen Betty, he's very nice to me.                 Amanda: Don't be shy. He's a very nice guy. He is very nice.


In [ ]:
print("Document: \n", xsum_dataset['test'][0]['document'])
print("Original: ", xsum_dataset['test'][0]['summary'])
print("Generated: ", xsum_test_subset[0]['generated_summary'])

Document: 
 Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.
Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.
The Welsh Government said more people than ever were getting help to address housing problems.
Changes to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.
Prison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.
However, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.
Andrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the need fo

**Observation:**
The tendency of distilbart-cnn-12-6 to prioritize the initial or topic sentences when summarizing is well-suited for news articles, where the most important information is often presented upfront. However, this approach can lead to less effective summaries for dialogues, where the key information might be spread throughout the conversation rather than at the beginning.

In [ ]:
from datasets import load_metric

rouge = load_metric('rouge')

/tmp/ipykernel_62/3688894951.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric('rouge')


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


In [ ]:
samsum_rouge_scores = rouge.compute(predictions=samsum_test["generated_summary"], references=samsum_test["summary"])
samsum_rouge_scores

{'rouge1': AggregateScore(low=Score(precision=0.19852457819821803, recall=0.5188277656594559, fmeasure=0.2741439248118298), mid=Score(precision=0.20483311155351527, recall=0.5311703338904473, fmeasure=0.28123381849264706), high=Score(precision=0.21143262976688673, recall=0.5440751384781168, fmeasure=0.2885093925677263)),
 'rouge2': AggregateScore(low=Score(precision=0.06773379413729942, recall=0.1827648144326898, fmeasure=0.09364599659874502), mid=Score(precision=0.07201805861042407, recall=0.19380442609893866, fmeasure=0.09911068428491915), high=Score(precision=0.07639766131120364, recall=0.20540839939003608, fmeasure=0.10455220555433374)),
 'rougeL': AggregateScore(low=Score(precision=0.14799006954422336, recall=0.3975048880773811, fmeasure=0.20604603955137418), mid=Score(precision=0.15300673375332763, recall=0.4090583462568975, fmeasure=0.2116341996844942), high=Score(precision=0.15822335421060707, recall=0.42072995068413677, fmeasure=0.21720324266344446)),
 'rougeLsum': AggregateSc

In [ ]:
xsum_rouge_scores = rouge.compute(predictions=xsum_test_subset["generated_summary"], references=xsum_test_subset["summary"])
xsum_rouge_scores

{'rouge1': AggregateScore(low=Score(precision=0.13809242829735938, recall=0.34039129316972755, fmeasure=0.1940320564045655), mid=Score(precision=0.1419369388321084, recall=0.3490022629282209, fmeasure=0.19887930742362098), high=Score(precision=0.14566128669170147, recall=0.3573691478284609, fmeasure=0.20395243836428395)),
 'rouge2': AggregateScore(low=Score(precision=0.02236879631041316, recall=0.05633766801378598, fmeasure=0.031492951491069315), mid=Score(precision=0.024351006398248734, recall=0.06144613125112401, fmeasure=0.034299928064285154), high=Score(precision=0.026390355269889462, recall=0.06676934632215757, fmeasure=0.03722454866855688)),
 'rougeL': AggregateScore(low=Score(precision=0.0903229556841106, recall=0.2234105485896511, fmeasure=0.12689450998755156), mid=Score(precision=0.09279964735857701, recall=0.22915599332281536, fmeasure=0.1300982338504887), high=Score(precision=0.09506844264959954, recall=0.23483599133171948, fmeasure=0.1332288348532887)),
 'rougeLsum': Aggreg

Step 4: Model Fine-Tuning

In [ ]:
from datasets import load_dataset

samsum_dataset = load_dataset("samsum")
xsum_dataset = load_dataset("EdinburghNLP/xsum")
samsum_train = samsum_dataset['train']
samsum_val = samsum_dataset['validation']
samsum_test = samsum_dataset['test']

In [ ]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

def preprocess_function(examples):
    inputs = tokenizer(examples['dialogue'], max_length=1024, truncation=True)
    targets = tokenizer(examples['summary'], max_length=128, truncation=True)
    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': targets['input_ids']
    }

tokenized_samsum_train = samsum_train.map(preprocess_function, batched=True)
tokenized_samsum_val = samsum_val.map(preprocess_function, batched=True)
tokenized_samsum_test = samsum_test.map(preprocess_function, batched=True)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [ ]:
def preprocess_xsum_function(examples):
    inputs = tokenizer(examples['document'], max_length=1024, truncation=True)
    targets = tokenizer(examples['summary'], max_length=128, truncation=True)
    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': targets['input_ids']
    }

tokenized_xsum_test = xsum_test.map(preprocess_xsum_function, batched=True)
tokenized_xsum_test_subset = tokenized_xsum_test.select(range(819))

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.02,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=500,
    fp16=True,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_samsum_train,
    eval_dataset=tokenized_samsum_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,1.646500,1.437786
2,1.290500,1.411307
3,1.116300,1.422624


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and retur

TrainOutput(global_step=2763, training_loss=1.331983082371262, metrics={'train_runtime': 4300.8965, 'train_samples_per_second': 10.276, 'train_steps_per_second': 0.642, 'total_flos': 2.764678008319181e+16, 'train_loss': 1.331983082371262, 'epoch': 3.0})

Step 5: Evaluation

In [ ]:
def generate_summary(example):
    inputs = torch.tensor(example['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(example['attention_mask']).unsqueeze(0).to(device)

    summary_ids = model.generate(inputs, attention_mask=attention_mask, max_length=128, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return {"generated_summary": summary}

samsum_test_with_summaries = tokenized_samsum_test.map(generate_summary)
xsum_test_with_summaries = tokenized_xsum_test_subset.map(generate_summary)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/tmp/ipykernel_62/1990374615.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(example['input_ids']).unsqueeze(0).to(device)
/tmp/ipykernel_62/1990374615.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(example['attention_mask']).unsqueeze(0).to(device)


In [ ]:
from evaluate import load

rouge = load("rouge")

In [ ]:
samsum_rouge_scores = rouge.compute(predictions=samsum_test_with_summaries["generated_summary"], references=samsum_test_with_summaries["summary"])

In [ ]:
xsum_rouge_scores = rouge.compute(predictions=xsum_test_with_summaries["generated_summary"], references=xsum_test_with_summaries["summary"])

In [ ]:
print("Evaluation Results on SAMSum Test Set:")
for key, value in samsum_rouge_scores.items():
    print(f"{key}: {value}")

Evaluation Results on SAMSum Test Set:
rouge1: 0.3988985196304326
rouge2: 0.20096704267215054
rougeL: 0.3050665531038751
rougeLsum: 0.3052674111900092


In [ ]:
print("Evaluation Results on XSum Test Subset:")
for key, value in xsum_rouge_scores.items():
    print(f"{key}: {value}")

Evaluation Results on XSum Test Subset:
rouge1: 0.20103645574255669
rouge2: 0.03815603060235087
rougeL: 0.13284899437336667
rougeLsum: 0.13285161261780748


In [ ]:
def display_examples(dataset_with_summaries, num_examples=5):
    for i in range(num_examples):
        print(f"Example {i+1}:")
        if 'dialogue' in dataset_with_summaries[i]:
            print(f"Dialogue: {dataset_with_summaries[i]['dialogue']}")
        print(f"Reference Summary: {dataset_with_summaries[i]['summary']}")
        print(f"Generated Summary: {dataset_with_summaries[i]['generated_summary']}")
        print("-" * 80)

print("SAMSum Test Set Examples:")
display_examples(samsum_test_with_summaries, num_examples=5)

SAMSum Test Set Examples:
Example 1:
Dialogue: Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
Reference Summary: Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.
Generated Summary: Larry called Betty last time they were at the park together. Hannah doesn't know Larry well. Amanda will text Larry.  .. Hannah and Amanda are going to meet at the end of the day. “Hannah’s not sure about it. 
--------------------------------------------------------------------------------
Example 2:
Dialogue: Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really fu

Step 6: Conclusion

### **Conclusion:**

The evaluation results demonstrate a significant improvement in the model's performance on dialogue summarization after fine-tuning on the SAMSum dataset. The ROUGE-1 score increased to 0.3989, and the ROUGE-2 score reached 0.2009, reflecting the model's enhanced ability to capture key information in dialogue-based content. This improvement underscores the effectiveness of domain-specific fine-tuning in enhancing summarization quality for dialogues.

However, the model showed only marginal improvement on the out-of-domain XSum test subset, with ROUGE-1 and ROUGE-2 scores remaining relatively low at 0.2010 and 0.0382, respectively. This suggests that while the model's performance in dialogue summarization improved significantly, its ability to generalize to more extreme extractive summarization tasks, like those presented in XSum, remains limited.

Overall, these results highlight the importance of fine-tuning for domain-specific tasks and the challenges associated with cross-domain generalization in text summarization models.